In [46]:
import pickle

In [20]:
groupjoin = pickle.load(open("data/groupjoin","rb"))

In [21]:
len(groupjoin)

5952

In [5]:
import pandas as pd

In [6]:
groupjoin = pickle.load(open("data/groupjoin","rb"))

In [7]:
len(groupjoin)

5952

In [15]:
len(groupjoin[1671174]),groupjoin[1671174][:10]

(1737,
 [(7467415, 1278656196000),
  (10290000, 1278656973000),
  (8978228, 1278677405000),
  (9041705, 1278681464000),
  (11413753, 1278685331000),
  (11738963, 1278686299000),
  (4323799, 1278688849000),
  (6657235, 1278697584000),
  (9061588, 1278698802000),
  (6008990, 1278699541000)])

In [16]:
ersvp = pickle.load(open("data/ersvp","rb"))

In [19]:
len(ersvp),ersvp['dlfrldysgbbc']

(294571,
 [(7918822, 1397910371000),
  (103016352, 1397757003000),
  (104586062, 1397918738000)])

In [20]:
new_grp_event = pickle.load(open("data/new_grp_event","rb"))

In [27]:
len(new_grp_event),len(new_grp_event[1671174]),new_grp_event[1671174][:10]

(5647,
 177,
 [('14013449', 1279029600000),
  ('14097177', 1282917600000),
  ('14704584', 1292598000000),
  ('16638356', 1298484000000),
  ('16908526', 1307111400000),
  ('24160391', 1310135400000),
  ('57096322', 1332608400000),
  ('56590142', 1335535200000),
  ('62400822', 1338559200000),
  ('67315922', 1340978400000)])

In [30]:
x = []
for i in new_grp_event :
    if i in groupjoin:
        x.append(i)

In [49]:
def GetValidGrps(new_grp_event,groupjoin) :
    return [i for i in new_grp_event if i in groupjoin]
            

In [36]:
len(x),x[:10]

(5319,
 [1671174,
  1638409,
  15859722,
  1769484,
  18645005,
  18448408,
  1703963,
  20185123,
  20447271,
  16711722])

In [50]:
len(GetValidGrps(new_grp_event,groupjoin))

5319

In [45]:
def make_social_graph_of_grp_members(grp_id,groupjoin,ersvp,new_grp_event) :
    member_set = set()
    for member,time in groupjoin[grp_id] :
        member_set.add(member)
    
    events_in_grp = []
    
    for event_id,time in new_grp_event[grp_id] :
        if event_id in ersvp:
            events_in_grp.append(event_id)
    edges = {}
    for event_id in events_in_grp :

        event_member_set = list(set([member for member,time in ersvp[event_id]  if member in member_set ]))
        event_member_set = sorted(event_member_set)
        for i in range(len(event_member_set)) :
            for j in range(i+1,len(event_member_set)):
                if (event_member_set[i],event_member_set[j]) in edges :
                    edges[(event_member_set[i],event_member_set[j])]+=1
                else :
                    edges[(event_member_set[i],event_member_set[j])]=1
    return list(member_set),[(i,j,edges[(i,j)]) for i,j in edges]
                
        
        
        
    
    

In [46]:
members,edges =  make_social_graph_of_grp_members(1671174,groupjoin,ersvp,new_grp_event)

In [47]:
(len(members),members[:10]),(len(edges),edges[:10])

((1737,
  [4677635,
   78503952,
   72146962,
   211894314,
   181026862,
   192176181,
   190709814,
   7921723,
   182566981,
   209363031]),
 (55199,
  [(5620833, 6657235, 1),
   (5620833, 7467415, 1),
   (5620833, 8978228, 2),
   (5620833, 9041705, 3),
   (5620833, 9061588, 1),
   (5620833, 9420414, 2),
   (5620833, 10290000, 1),
   (5620833, 11350459, 2),
   (5620833, 11413753, 2),
   (5620833, 11738963, 5)]))

In [48]:
def make_member_level_features(grp_id,groupjoin,ersvp,new_grp_event) :
    members,edges =  make_social_graph_of_grp_members(grp_id,groupjoin,ersvp,new_grp_event)
    adjacency_list = {}
    for member in members :
        adjacency_list[member] = []
    for i,j,w in edges :
        adjacency_list[i].append((j,w))
        adjacency_list[j].append((i,w))
    

In [51]:
def get_memberId_to_groups(valid_grps,groupjoin) :
    memberId_to_groups = {}
    for grp in valid_grps :
        for member , time in groupjoin[grp] :
            if member in  memberId_to_groups :
                memberId_to_groups[member].append(grp)
            else :
                memberId_to_groups[member] = [grp]
    return memberId_to_groups
    
    

In [54]:
def get_grp_to_memberId_to_attended_events(grp_to_valid_events,groupjoin,ersvp):
    grp_to_memberId_to_attended_events = {}
    for grp in grp_to_valid_events :
        memberId_to_attended_events = {}
        for event,_ in grp_to_valid_events[grp] :
            for member,time in ersvp[event] :
                if member in memberId_to_attended_events :
                    memberId_to_attended_events.append(event)
                else :
                    memberId_to_attended_events = [event]
        grp_to_memberId_to_attended_events[grp] = memberId_to_attended_events
    return grp_to_memberId_to_attended_events
                
    
    
    

In [55]:
def get_grp_to_valid_events(valid_grps,new_grp_event,ersvp):
    grp_to_valid_events = {}
    for i in valid_grps :
        for j,_ in new_grp_event[i] :
            if j in ersvp :
                grp_to_valid_events[i].append(j)
    return grp_to_valid_events

In [57]:
from sklearn.decomposition import NMF

In [58]:
mat = [[2,3,5,6],[45,89,4,36]]

In [64]:
model = NMF(n_components =3 )

In [65]:
model.fit(mat)

/home/sanket/miniconda3/envs/data_science/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:315: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  "'nndsvda' in 1.1 (renaming of 0.26)."), FutureWarning)


NMF(n_components=3)

In [66]:
nmf_features = model.transform(mat)

In [67]:
nmf_features

array([[ 0.22759124,  0.73157561,  0.77159923],
       [ 8.60254867, 14.46340326,  0.        ]])

In [68]:
model.components_

array([[1.32996650e+00, 7.40893980e+00, 3.06939420e-03, 2.87414106e+00],
       [2.31995587e+00, 1.74713467e+00, 2.74681609e-01, 7.79672477e-01],
       [0.00000000e+00, 4.61814381e-02, 6.21817075e+00, 6.18961015e+00]])

In [69]:
import pickle

In [72]:
elat = pickle.load(open("data/elat","rb"))
elon = pickle.load(open("data/elon","rb"))

In [73]:
len(elat),len(elon)

(458087, 458087)

In [75]:
list(elat.keys())[:10]

['dvlgvlywjbxb',
 'dlfrldysgbbc',
 'hrkzllyvfbjc',
 '170374282',
 '228592712',
 'njvctlyvlbhc',
 'dpqbgcyrnbqb',
 'dxzpffywjbfc',
 'dlzvbfyrcbtb',
 '161926702']

In [77]:
elat['dvlgvlywjbxb'],elon['dvlgvlywjbxb']

(41.908699, -87.6819)

In [78]:
from pathlib import Path

In [100]:
len([str(x).split('/')[-1].split('.')[0] for x in Path("member_level_features").iterdir()])

5320

In [88]:
[x for x in Path("member_level_features").iterdir()][:10]

[PosixPath('member_level_features/1671174.csv'),
 PosixPath('member_level_features/1638409.csv'),
 PosixPath('member_level_features/15859722.csv'),
 PosixPath('member_level_features/1769484.csv'),
 PosixPath('member_level_features/18645005.csv'),
 PosixPath('member_level_features/18448408.csv'),
 PosixPath('member_level_features/1703963.csv'),
 PosixPath('member_level_features/20185123.csv'),
 PosixPath('member_level_features/20447271.csv'),
 PosixPath('member_level_features/16711722.csv')]

In [89]:
df = pd.read_csv("member_level_features/1671174.csv")

In [90]:
df

,member,group_id,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12
0,4677635,1671174,72,1,13,0.918296,3.700440,1.584963,374.916667,33.944444,13.013889,1.724832,2.668147,2.092105
1,78503952,1671174,0,0,23,0.000000,4.175736,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,72146962,1671174,0,0,11,1.000000,3.459432,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,211894314,1671174,0,0,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,181026862,1671174,0,0,6,1.000000,2.584963,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732,15802341,1671174,60,1,3,0.000000,1.584963,0.000000,388.300000,38.966667,10.266667,1.659389,2.472409,2.008393
1733,134496232,1671174,81,1,2,0.000000,1.000000,0.000000,338.407407,28.444444,14.123457,1.566956,2.668083,2.028720
1734,201498614,1671174,0,0,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1735,62038012,1671174,0,0,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [91]:
x = {}

In [92]:
x[(45,89)] =456

In [93]:
x[(1,-1)] =12

In [95]:
x[(1,-0)]

KeyError: (1, 0)

In [96]:
groupjoin = pickle.load(open("data/groupjoin","rb"))
new_grp_event = pickle.load(open("data/new_grp_event","rb"))
ersvp = pickle.load(open("data/ersvp","rb"))

In [97]:
len(groupjoin)

5952

In [98]:
len(new_grp_event)

5647

In [99]:
len(ersvp)

294571

In [1]:
from pathlib import Path
y= [x for x in Path("group_level_features").iterdir()]

In [2]:
len(y)

2135

In [4]:
import torch

In [5]:
print(torch.__version__)

1.9.0


In [6]:
a = torch.full((2,3),45)

In [7]:
a

tensor([[45, 45, 45],
        [45, 45, 45]])

In [13]:
b= torch.full((10,3),-1)

In [14]:
torch.cat((a,b))

tensor([[45, 45, 45],
        [45, 45, 45],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1]])

In [23]:
# Create tensors.
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True)
b = torch.tensor(5., requires_grad=True)
x, w, b

(tensor(3.), tensor(4., requires_grad=True), tensor(5., requires_grad=True))

In [27]:
# Arithmetic operations
y = w * x + b
y

tensor(17., grad_fn=<AddBackward0>)

In [30]:
# Compute derivatives
y.backward()

In [31]:
# Display gradients
print('dy/dx:', x.grad)
print('dy/dw:', w.grad)
print('dy/db:', b.grad)

dy/dx: None
dy/dw: tensor(3.)
dy/db: tensor(1.)


In [28]:
y = w*x+b+10
y

tensor(27., grad_fn=<AddBackward0>)

In [43]:
n = 5
x = torch.full((n,1),torch.tensor(3.,requires_grad=True))
x.requires_grad_(True)
w = torch.full((n,n),1.)
b = torch.full((n,1),20.)

TypeError: full() received an invalid combination of arguments - got (tuple, Tensor), but expected one of:
 * (tuple of ints size, Number fill_value, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, Number fill_value, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [41]:
y= w*x+b

In [38]:
print('dy/dx:', x.grad)
print('dy/dw:', w.grad)
print('dy/db:', b.grad)

dy/dx: None
dy/dw: None
dy/db: None


In [42]:
y.backward()

RuntimeError: grad can be implicitly created only for scalar outputs

In [52]:
a = torch.tensor([[1,2,3],[4,5,6]])

In [53]:
a

tensor([[1, 2, 3],
        [4, 5, 6]])

In [54]:
a.shape

torch.Size([2, 3])

In [56]:
a[1],a[1].shape

(tensor([4, 5, 6]), torch.Size([3]))

In [7]:
import torch


In [10]:
torch.full((1,6),1)

tensor([[1, 1, 1, 1, 1, 1]])

In [17]:
a=torch.tensor([1,23,56])
a,a.shape

(tensor([ 1, 23, 56]), torch.Size([3]))

In [16]:
b=torch.tensor([[1,3,2]])
b,b.shape

(tensor([[1, 3, 2]]), torch.Size([1, 3]))

In [19]:
a@b.t()

tensor([182])

In [20]:
a@b

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x3 and 1x3)

In [21]:
a. @ b

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x3 and 1x3)

In [23]:
a,a.t()

(tensor([ 1, 23, 56]), tensor([ 1, 23, 56]))

In [25]:
torch.transpose(a,1)

TypeError: transpose() received an invalid combination of arguments - got (Tensor, int), but expected one of:
 * (Tensor input, int dim0, int dim1)
 * (Tensor input, name dim0, name dim1)


In [36]:
 torch.reshape(a,(1,3))

tensor([[ 1, 23, 56]])

In [30]:
import numpy as np

In [31]:
x = np.array([[1,2,30],[15,25,350]])

In [32]:
x

array([[  1,   2,  30],
       [ 15,  25, 350]])

In [34]:
x[0],x[0].shape

(array([ 1,  2, 30]), (3,))

In [35]:
torch.tensor(x[0])

tensor([ 1,  2, 30])

In [44]:
a= torch.tensor([[ 1,  2, 30]])

In [43]:
b= torch.tensor([[ 11,  12]])

In [45]:
torch.cat((a,b),1)

tensor([[ 1,  2, 30, 11, 12]])

In [47]:
groupjoin = pickle.load(open("data/groupjoin","rb"))

In [48]:
import pickle

In [49]:
tag_new_slide = pickle.load(open("data/tag_new_slide1","rb"))

In [82]:
list(tag_new_slide)[2+0:2+10]

[1187853,
 11870222,
 19513360,
 1826841,
 19587098,
 1703963,
 18489374,
 1368097,
 16621602,
 1671174]

In [51]:
tag_new_slide[18497538]

[-1, -1, -1]

In [52]:
new_grp_event = pickle.load(open("data/new_grp_event","rb"))

In [54]:
new_grp_event[18497538],len(new_grp_event[18497538])

([('221115653', 1428796800000),
  ('221800730', 1430611200000),
  ('222518168', 1433635200000),
  ('223483309', 1435449600000),
  ('224830260', 1441497600000),
  ('225833501', 1444521600000),
  ('225833509', 1446944400000),
  ('225833520', 1449363600000),
  ('229209253', 1457226000000),
  ('229209293', 1459641600000),
  ('230832459', 1462665600000)],
 11)

In [57]:
tag_new_slide[19841030],len(tag_new_slide[19841030])

([-1, -1, -1, -1], 4)

In [56]:
new_grp_event[19841030],len(new_grp_event[19841030])

([('230378231', 1461027600000),
  ('230495467', 1461805200000),
  ('232202645', 1467387000000),
  ('232202449', 1468364400000),
  ('231777396', 1468364400000),
  ('232569037', 1469296800000),
  ('232423279', 1469296800000),
  ('234230808', 1475542800000),
  ('234590613', 1475931600000),
  ('234594790', 1476536400000),
  ('234766540', 1477227600000),
  ('235406723', 1478959200000)],
 12)

In [58]:
sum([len(new_grp_event[i])-9 for i in new_grp_event])

379583

In [64]:
min([(len(new_grp_event[i])-9,i) for i in new_grp_event])

(-8, 1274964)

In [63]:
tag_new_slide[1274964]

KeyError: 1274964

In [65]:
len(tag_new_slide),len(new_grp_event)

(2088, 5647)

In [76]:
min([(len(new_grp_event[i])-10,i) for i in tag_new_slide])

(0, 1417721)

In [69]:
sum([len(new_grp_event[i])-9 for i in tag_new_slide])

309644

In [70]:
groupjoin = pickle.load(open("data/groupjoin","rb"))

In [71]:
list(groupjoin)[:10]

[1671174,
 21004296,
 1638409,
 15859722,
 1769484,
 18645005,
 18448408,
 1703963,
 20185123,
 20447271]

In [73]:
len(groupjoin[1671174])

1737

In [75]:
len(groupjoin[1671174])

1737

In [79]:
new_grp_event[1417721]

[('10033835', 1240084800000),
 ('10214615', 1240686000000),
 ('10214617', 1241895600000),
 ('10214618', 1242504000000),
 ('10214619', 1243108800000),
 ('10266314', 1243713600000),
 ('10375834', 1244318400000),
 ('10375835', 1244923200000),
 ('10427076', 1245528000000),
 ('10478778', 1246132800000)]

In [80]:
tag_new_slide[1417721]

[-1, -1]

In [83]:
print(*[1,13,23])

1 13 23


In [84]:
print(1,12,15)

1 12 15
